# Three point problem

Imagine that you are on a geological mapping campaign and that you observe a geological boundary at three points. You assume that the boundary is a flat plane and you would like to know the strike/ dip of this plane. 

This is a classical spatial interpolation problem (often called the "three-point-problem") and there are several possibilities to obtain a result (and maybe you discussed some in your geological mapping classes). You will here use the methods of linear interpolation that we discussed during the lecture.

We denote the positions of our observation points (i.e. the $x$-, $y$-coordinates) as:

$$x_i, y_i \mbox{ for } i \in {0,1,2}$$

And the $z$ value of our observation is the dependent value:

$$z_i = f(x_i,y_i) \mbox{ for } i \in {0,1,2}$$

We can then write the equations for first-order linear interpolation as:

$$z_0 = a_0 + a_1 x_0 + a_2 y_0$$
$$z_1 = a_0 + a_1 x_1 + a_2 y_1$$
$$z_2 = a_0 + a_1 x_2 + a_2 y_2$$

Or in matrix form as:

$$\left[ \begin{array}{ccc}
1 & x_0 & y_0 \\
1 & x_1 & y_1 \\
1 & x_2 & y_2 \end{array} \right]
\left\{ \begin{array}{ccc}
a_0 \\
a_1 \\
a_2 \end{array} \right\}
=
\left\{ \begin{array}{ccc}
z_0 \\
z_1 \\
z_2 \end{array} \right\}
$$

We now need a method to solve this equation for the vector with the unknown $a_i$'s. One convenient method to solve these types of equations is implemented in the `numpy` method `np.linalg.solve()`. For example, if the matrix is stored in variable `B`, and the z-values in an array variable `z`, and the a-values in variable `a` so that we can write the equation in the form:

$$B a = z$$

Then we can obtain the results of `a` as:

```python
(a_0, a_1, a_2) = np.linalg.solve(B, z)
```

Going back to the equations at the beginning, you can see that we now have the slope values in $x$- and $y$- direction (partial derivatives for each axis) stored in the variables $a_1$ and $a_2$. We can use these results to obtain the structural geological (dip-direction/ dip) values using the following equations:

```python
maxslope = np.sqrt(a_1**2 + a_2**2)
dip = np.arctan(maxslope) / np.pi * 180.
dip_direction = np.abs(np.arctan2(-a_1, -a_2)) /  np.pi * 180.
```

All of these aspects are implemented in the following module and can be called with the function `three_points_to_plane()`:

In [1]:
import os, sys
sys.path.append(r"..")

In [2]:
import struct_geo
reload(struct_geo)

<module 'struct_geo' from '../struct_geo.pyc'>

In [334]:
# test your implementation:
p_0 = (0, 0, 0)
p_1 = (1, 0, 0)
p_2 = (0, 1, -1)

# from dome
import numpy as np
# p_0 = np.array([2478083., 615163., 385.])
# p_1 = np.array([2477890., 615172., 439.])
# p_2 = np.array([2477825., 615029., 386.])


p_0 = np.array([615163, 2478083.5, 385])
p_1 = np.array([615172, 2477890, 439])
p_2 = np.array([615029, 2477825, 385])

P = np.vstack([p_0, p_1, p_2])

In [335]:
P

array([[  6.15163000e+05,   2.47808350e+06,   3.85000000e+02],
       [  6.15172000e+05,   2.47789000e+06,   4.39000000e+02],
       [  6.15029000e+05,   2.47782500e+06,   3.85000000e+02]])

In [337]:
P[:,0] -= np.min(P[:,0])
P[:,1] -= np.min(P[:,1])
P[:,2] -= np.min(P[:,2])
P

array([[ 134. ,  258.5,    0. ],
       [ 143. ,   65. ,   54. ],
       [   0. ,    0. ,    0. ]])

## Determine plane orientation

In [341]:
dip, dip_direction = struct_geo.calculate_dip_direction_dip(P[0,:], P[1,:], P[2,:])
print("The correspoding plane is (%03d/%02d)" % (dip_direction, dip))

The correspoding plane is (062/29)


## Plot plane

In [180]:
struct_geo.plot_three_point(p_0, p_1, p_2)

In [193]:
P = P/np.max(P)

In [194]:
P

array([[  2.48241433e-01,   1.00000000e+00,   1.55361996e-04],
       [  2.48245065e-01,   9.99921915e-01,   1.77153030e-04],
       [  2.48187359e-01,   9.99895686e-01,   1.55361996e-04]])

In [338]:
v1 = P[0,:] - P[1,:]
v2 = P[1,:] - P[2,:]

In [289]:
cp = [v1[1] * v2[2] - v1[2] * v2[1],
      v1[2] * v2[0] - v1[0] * v2[2],
      v1[0] * v2[1] - v1[1] * v2[0]]

In [339]:
normal_vec = np.cross(v1, v2)
normal_vec

array([ 13959. ,  -7236. , -28255.5])

In [340]:
(a_1, a_2, a_0) = normal_vec

maxslope = np.sqrt(a_1 ** 2 + a_2 ** 2)
dip = np.arctan(maxslope) / np.pi * 180.
dip_direction = np.abs(np.arctan2(-a_1, a_2)) / np.pi * 180.
print(dip_direction, dip)

(117.40110535941861, 89.996355930525766)


In [303]:
def dip(normal_vec):
    return np.arctan(normal_vec[2]) / np.pi * 180.

In [304]:
dip(normal_vec)

45.0

In [305]:
# determine dip direction
def dip_dir(normal_vec):
    # +/+
    if normal_vec[0] >= 0 and normal_vec[1] > 0:
        return np.arctan(normal_vec[0]/normal_vec[1]) / np.pi * 180.
    # border cases where arctan not defined:
    elif normal_vec[0] > 0 and normal_vec[1] == 0:
        return 90
    elif normal_vec[0] < 0 and normal_vec[1] == 0:
        return 270
    # +-/-
    elif normal_vec[1] < 0:
        return 180 + np.arctan(normal_vec[0]/normal_vec[1]) / np.pi * 180.
    # -/-
    elif normal_vec[0] < 0 and normal_vec[1] >= 0:
        return 360 + np.arctan(normal_vec[0]/normal_vec[1]) / np.pi * 180.
#    elif normal_vec[1] == 0:
#        return 90



In [306]:
dip_dir(normal_vec)

0.0

In [295]:
P

array([[ 0,  0,  0],
       [ 1,  0,  0],
       [ 0,  1, -1]])

In [217]:
P = P/np.max(P)

In [218]:
dip, dip_direction = struct_geo.calculate_dip_direction_dip(P[0,:], P[1,:], P[2,:])
print("The correspoding plane is (%03d/%02d)" % (dip_direction, dip))

The correspoding plane is (062/29)


In [230]:
B = np.array([[1, P[0,0], P[0,1]],
              [1, P[1,0], P[1,1]],
              [1, P[2,0], P[2,1]]])
              
z = np.array([P[0,2], P[1,2], P[2,2]])

(a_0, a_1, a_2) = np.linalg.solve(B, z)

maxslope = np.sqrt(a_1**2 + a_2**2)
dip_val = np.arctan(maxslope) / np.pi * 180.
dip_direction_val = np.abs(np.arctan2(-a_1, -a_2)) /  np.pi * 180.
print(dip_direction_val, dip_val)

(62.598894640562584, 29.094133772175024)


In [244]:
nv = np.array([a_0, a_1, a_2])

In [245]:
dip(nv)

104.83828516458233

In [246]:
dip_dir(nv)

15.13351226003142

In [247]:
nv

array([ 0.13360895,  0.49402771, -0.25609173])

In [315]:
(a_1, a_2, a_0) = normal_vec

maxslope = np.sqrt(a_1 ** 2 + a_2 ** 2)
dip = np.arctan(maxslope) / np.pi * 180.
dip_direction = np.abs(np.arctan2(-a_1, a_2)) / np.pi * 180.


In [316]:
dip, dip_direction

(45.0, 0.0)

In [317]:
normal_vec

array([0, 1, 1])